# Testing irrep construction

Given state $\ket{v}$, 1D irrep $T$ of order $N$ (i.e. such that 
$$
\{\ket{v}, T\ket{v}, T^2\ket{v}, ... , T^{N-1}\ket{v}\}
$$
is a subspace of dimension $N$), straightforward to construct Bloch waves

$$
\ket{k} := \sum_{n=0}^{N-1} e^{i k n} T^{n}\ket{v}
$$

where $k = 2\pi m/N$ for some integer $m$.



In [286]:
import numpy as np
import bit_tools
import bisect
from sympy.combinatorics import PermutationGroup, Permutation

basis = sorted([0b0011, 0b0101, 0b0110, 0b1010, 0b1100, 0b1001])


In [287]:
import itertools

def perm_order(P):
    # expects P to be a permutation written as a list
    order = 1

    P = np.array(P)
    y = P.copy()
    while any(i != x for i, x in enumerate(y) ):
        order += 1
        y = P[y]
    return order


In [289]:
# def idperm_like(g):
#     return np.array([i for i,_ in enumerate(g)])


# def make_perm_powers(perms):

#     idperm = idperm_like(perms[0])
#     # pre-calculate nth powers of the perms
    
#     assert all(len(P) == len(idperm) for P in perms), "all permutations must have same length"

#     perm_powers = []
#     for P in perms:
#         x = idperm
#         power_list = [idperm]
#         while True:
#             x = P[x]
#             if np.all(x == idperm):
#                 break
#             power_list.append(x)
#         perm_powers.append(power_list)

#     return perm_powers




In [297]:


def partition_basis(blist, group:PermutationGroup, verify_basis=True):
    orbits = []    
    # the basis elements already in an orbit
    covered_elems = []

    idx = 0
    while idx < len(blist):
        seed_state = blist[idx]

        J = bisect.bisect_left(covered_elems, seed_state)
        if J < len(covered_elems) and covered_elems[J] == seed_state:
            # we have already hit this
            idx+= 1
            continue
            

        curr_orbit = []
        for g in group.elements:
                
            state = bit_tools.bitperm(g.array_form, seed_state)
            idx_to_insert = bisect.bisect_left(covered_elems, state)

            if verify_basis:
                J = bisect.bisect_left(blist, state)
                if blist[J] != state:
                    raise ValueError("Basis is not closed under the symmetry operation")
            
            if idx_to_insert >= len(covered_elems) or covered_elems[idx_to_insert] != state:
                covered_elems.insert(idx_to_insert, state)
            
                curr_orbit.append(state)
            
        

        orbits.append(curr_orbit)
        idx+= 1

    return orbits

In [285]:
class RepDecomposition:
    def __init__(self, basis_list):
        self.basis_list.append

SyntaxError: incomplete input (1797845655.py, line 3)

In [310]:
P = PermutationGroup([Permutation(x) for x in [[1,2,0,3], [1,0,2,3]]])

In [313]:
# G = generate_Abelian([0,1,3,2], [1,0,2,3])
res = partition_basis(basis,  P)

In [314]:
def fmt_bin(x):
    return f"{x:06b}"

for r in res:
    print("[")
    for k in r:
        print(f"{k:06b} ")
    print("]")

[
000011 
000110 
000101 
]
[
001001 
001010 
001100 
]


In [323]:
P.conjugacy_classes()[1]

{Permutation(3)(0, 1, 2), Permutation(3)(0, 2, 1)}

In [319]:
import sympy as syp

def decompose_orbit(orbit, characters, cc_list):
    # :param characters, cc_list -> matching characters and conjugacy classes
    # step 1: compute perm group rep
    M = syp.zeros(len(orbit),len(orbit))
    print(M)
    assert len(characters) == len(G.conjugacy_classes)
    cc_repr_matrices = 
    for cc in G.conjugacy_classes():
        tmp = syp.zeros(len(orbit),len(orbit))
        for state in orbit:
            bit_tools.bitperm(
        

decompose_orbit(res[0], P)

Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])


In [308]:
# class CharacterTable:
#     def __init__(self, G:PermutationGroup):
#         self.G = G
#         self.


# def decompose_orbits(orbit_set, ct:CharacterTable):

In [279]:
res[0]

[np.int64(3)]